In [ ]:
# Install the necessary dependencies

import os
import sys
!{sys.executable} -m pip install --quiet pandas scikit-learn numpy matplotlib jupyterlab_myst ipython

---
license:
    code: MIT
    content: CC-BY-4.0
github: https://github.com/ocademy-ai/machine-learning
venue: By Ocademy
open_access: true
bibliography:
  - https://raw.githubusercontent.com/ocademy-ai/machine-learning/main/open-machine-learning-jupyter-book/references.bib
---


# Pandas

Pandas is a fast, powerful, flexible and easy-to-use open-source data analysis and manipulation tool, built on top of the Python programming language.

## Introducing Pandas objects

We’ll start with a quick, non-comprehensive overview of the fundamental data structures in Pandas to get you started. The fundamental behavior about data types, indexing, axis labeling, and alignment apply across all of the objects. To get started, import NumPy and load Pandas into your namespace:

In [ ]:
import numpy as np
import pandas as pd

### Series

`Series` is a one-dimensional labeled array capable of holding any data type (integers, strings, floating point numbers, Python objects, etc.). The axis labels are collectively referred to as the **index**. 

Here, `data` can be many different things:

- a Python dict
- an ndarray
- a scalar value (like 5)


The passed **index** is a list of axis labels. Thus, this separates into a few cases depending on what the **data is**:

#### Create a Series

##### From ndarray

If `data` is an ndarray, **index** must be the same length as the **data**. If no index is passed, one will be created having values `[0, ..., len(data) - 1]`.

In [ ]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])

In [ ]:
s

In [ ]:
s.index

In [ ]:
pd.Series(np.random.randn(5))

Pandas supports non-unique index values. If an operation that does not support duplicate index values is attempted, an exception will be raised at that time.

##### From dict
`Series` can be instantiated from dicts:

In [ ]:
d = {"b": 1, "a": 0, "c": 2}

In [ ]:
pd.Series(d)

If an index is passed, the values in data corresponding to the labels in the index will be pulled out.

In [ ]:
d = {"a": 0.0, "b": 1.0, "c": 2.0}

In [ ]:
pd.Series(d)

In [ ]:
pd.Series(d, index=["b", "c", "d", "a"])

NaN (not a number) is the standard missing data marker used in Pandas.

##### From scalar value

If `data` is a scalar value, an index must be provided. The value will be repeated to match the length of **index**.

In [ ]:
pd.Series(5.0, index=["a", "b", "c", "d", "e"])

#### Series is ndarray-like

`Series` acts very similarly to a `ndarray` and is a valid argument to most NumPy functions. However, operations such as slicing will also slice the index.

In [ ]:
s[0]

In [ ]:
s[:3]

In [ ]:
s[s > s.median()]

In [ ]:
s[[4, 3, 1]]

In [ ]:
np.exp(s)

Like a NumPy array, a Pandas Series has a single `dtype`.

In [ ]:
s.dtype

If you need the actual array backing a `Series`, use `Series.array`.

In [ ]:
s.array

While `Series` is ndarray-like, if you need an actual ndarray, then use `Series.to_numpy()`.

In [ ]:
s.to_numpy()

Even if the `Series` is backed by an `ExtensionArray`, `Series.to_numpy()` will return a NumPy ndarray.

#### Series is dict-like

A `Series` is also like a fixed-size dict in that you can get and set values by index label:

In [ ]:
s["a"]

In [ ]:
s["e"] = 12.0

In [ ]:
s

In [ ]:
"e" in s

In [ ]:
"f" in s

Using the `Series.get()` method, a missing label will return None or specified default:

In [ ]:
s.get("f")

In [ ]:
s.get("f", np.nan)

These labels can also be accessed by `attribute`.

#### Vectorized operations and label alignment with Series

When working with raw NumPy arrays, looping through value-by-value is usually not necessary. The same is true when working with `Series` in Pandas. `Series` can also be passed into most NumPy methods expecting an ndarray.

In [ ]:
s + s

In [ ]:
s * 2

In [ ]:
np.exp(s)

A key difference between `Series` and ndarray is that operations between `Series` automatically align the data based on the label. Thus, you can write computations without giving consideration to whether the `Series` involved have the same labels.

In [ ]:
s[1:] + s[:-1]

The result of an operation between unaligned `Series` will have the **union** of the indexes involved. If a label is not found in one `Series` or the other, the result will be marked as missing `NaN`. Being able to write code without doing any explicit data alignment grants immense freedom and flexibility in interactive data analysis and research. The integrated data alignment features of the Pandas data structures set Pandas apart from the majority of related tools for working with labeled data.

In general, we chose to make the default result of operations between differently indexed objects yield the **union** of the indexes in order to avoid loss of information. Having an index label, though the data is missing, is typically important information as part of a computation. You of course have the option of dropping labels with missing data via the `dropna` function.

#### Name attribute

`Series` also has a `name` attribute:

In [ ]:
s = pd.Series(np.random.randn(5), name="something")

In [ ]:
s

In [ ]:
s.name

The `Series` `name` can be assigned automatically in many cases, in particular, when selecting a single column from a `DataFrame`, the `name` will be assigned the column label.

You can rename a `Series` with the `pandas.Series.rename()` method.

In [ ]:
s2 = s.rename("different")

In [ ]:
s2.name

Note that `s` and `s2` refer to different objects.

### DataFrame

`DataFrame` is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a `dict` of `Series` objects. It is generally the most commonly used Pandas object. Like `Series`, `DataFrame` accepts many different kinds of input:

- Dict of 1D ndarrays, lists, dicts, or `Series`
- 2-D `numpy.ndarray`
- Structured or record ndarray
- A `Series`
- Another `DataFrame`

Along with the data, you can optionally pass **index** (row labels) and **columns** (column labels) arguments. If you pass an index and / or columns, you are guaranteeing the index and / or columns of the resulting `DataFrame`. Thus, a `dict` of Series plus a specific index will discard all data not matching up to the passed index.

If axis labels are not passed, they will be constructed from the input data based on common sense rules.

#### Create a Dataframe

##### From dict of `Series` or dicts

The resulting **index** will be the **union** of the indexes of the various Series. If there are any nested dicts, these will first be converted to Series. If no columns are passed, the columns will be the ordered list of `dict` keys.

In [ ]:
d = {
    "one": pd.Series([1.0, 2.0, 3.0], index=["a", "b", "c"]),
    "two": pd.Series([1.0, 2.0, 3.0, 4.0], index=["a", "b", "c", "d"]),
}

In [ ]:
df = pd.DataFrame(d)

In [ ]:
df

In [ ]:
pd.DataFrame(d, index=["d", "b", "a"])

In [ ]:
pd.DataFrame(d, index=["d", "b", "a"], columns=["two", "three"])

The row and column labels can be accessed respectively by accessing the **index** and **columns** attributes:

When a particular set of columns is passed along with a dict of data, the passed columns override the keys in the dict.

In [ ]:
df.index

In [ ]:
df.columns

##### From dict of ndarrays / lists

The ndarrays must all be the same length. If an index is passed, it must also be the same length as the arrays. If no index is passed, the result will be `range(n)`, where `n` is the array length.

In [ ]:
d = {"one": [1.0, 2.0, 3.0, 4.0], "two": [4.0, 3.0, 2.0, 1.0]}

In [ ]:
pd.DataFrame(d)

In [ ]:
pd.DataFrame(d, index=["a", "b", "c", "d"])

##### From structured or record array

This case is handled identically to a dict of arrays.

In [ ]:
data = np.zeros((2,), dtype=[("A", "i4"), ("B", "f4"), ("C", "a10")])

In [ ]:
data[:] = [(1, 2.0, "Hello"), (2, 3.0, "World")]

In [ ]:
pd.DataFrame(data)

In [ ]:
pd.DataFrame(data, index=["first", "second"])

In [ ]:
pd.DataFrame(data, columns=["C", "A", "B"])

DataFrame is not intended to work exactly like a 2-dimensional NumPy ndarray.

##### From a list of dicts

In [ ]:
data2 = [{"a": 1, "b": 2}, {"a": 5, "b": 10, "c": 20}]

In [ ]:
pd.DataFrame(data2)

In [ ]:
pd.DataFrame(data2, index=["first", "second"])

In [ ]:
pd.DataFrame(data2, columns=["a", "b"])

##### From a dict of tuples

You can automatically create a MultiIndexed frame by passing a tuples dictionary.

In [ ]:
pd.DataFrame(
    {
        ("a", "b"): {("A", "B"): 1, ("A", "C"): 2},
        ("a", "a"): {("A", "C"): 3, ("A", "B"): 4},
        ("a", "c"): {("A", "B"): 5, ("A", "C"): 6},
        ("b", "a"): {("A", "C"): 7, ("A", "B"): 8},
        ("b", "b"): {("A", "D"): 9, ("A", "B"): 10},
    }
)

##### From a Series

The result will be a DataFrame with the same index as the input Series, and with one column whose name is the original name of the Series (only if no other column name provided).

In [ ]:
ser = pd.Series(range(3), index=list("abc"), name="ser")

In [ ]:
pd.DataFrame(ser)

##### From a list of namedtuples

The field names of the first `namedtuple` in the list determine the columns of the `DataFrame`. The remaining namedtuples (or tuples) are simply unpacked and their values are fed into the rows of the `DataFrame`. If any of those tuples is shorter than the first `namedtuple` then the later columns in the corresponding row are marked as missing values. If any are longer than the first `namedtuple` , a `ValueError` is raised.

In [ ]:
from collections import namedtuple

In [ ]:
Point = namedtuple("Point", "x y")

In [ ]:
pd.DataFrame([Point(0, 0), Point(0, 3), (2, 3)])

In [ ]:
Point3D = namedtuple("Point3D", "x y z")

In [ ]:
pd.DataFrame([Point3D(0, 0, 0), Point3D(0, 3, 5), Point(2, 3)])

##### From a list of dataclasses

Data Classes as introduced in PEP557, can be passed into the DataFrame constructor. Passing a list of dataclasses is equivalent to passing a list of dictionaries.

Please be aware, that all values in the list should be dataclasses, mixing types in the list would result in a `TypeError`.

In [ ]:
from dataclasses import make_dataclass

In [ ]:
Point = make_dataclass("Point", [("x", int), ("y", int)])

In [ ]:
pd.DataFrame([Point(0, 0), Point(0, 3), Point(2, 3)])

#### Column selection, addition, deletion

You can treat a `DataFrame` semantically like a dict of like-indexed `Series` objects. Getting, setting, and deleting columns works with the same syntax as the analogous dict operations:

In [ ]:
df

In [ ]:
df["one"]

In [ ]:
df["three"] = df["one"] * df["two"]

In [ ]:
df["flag"] = df["one"] > 2

In [ ]:
df

Columns can be deleted or popped like with a dict:

In [ ]:
del df["two"]

In [ ]:
three = df.pop("three")

In [ ]:
df

When inserting a scalar value, it will naturally be propagated to fill the column:

In [ ]:
df["foo"] = "bar"

In [ ]:
df

When inserting a `Series` that does not have the same index as the `DataFrame`, it will be conformed to the DataFrame’s index:

In [ ]:
df["one_trunc"] = df["one"][:2]

In [ ]:
df

You can insert raw ndarrays but their length must match the length of the DataFrame’s index.

By default, columns get inserted at the end. `DataFrame.insert()` inserts at a particular location in the columns:

In [ ]:
df.insert(1, "bar", df["one"])

In [ ]:
df

#### Assigning new columns in method chains

DataFrame has an `assign()` method that allows you to easily create new columns that are potentially derived from existing columns.

In [ ]:
iris = pd.read_csv("../../assets/data/iris.csv")

In [ ]:
iris.head()

In [ ]:
iris.assign(sepal_ratio=iris["SepalWidth"] / iris["SepalLength"]).head()

In the example above, we inserted a precomputed value. We can also pass in a function of one argument to be evaluated on the DataFrame being assigned to.

In [ ]:
iris.assign(sepal_ratio=lambda x: (x["SepalWidth"] / x["SepalLength"])).head()

`assign()` **always** returns a copy of the data, leaving the original DataFrame untouched.

Passing a callable, as opposed to an actual value to be inserted, is useful when you don’t have a reference to the DataFrame at hand. This is common when using `assign()` in a chain of operations. For example, we can limit the DataFrame to just those observations with a Sepal Length greater than 5, calculate the ratio, and plot:

In [ ]:
(
    iris.query("SepalLength > 5")
    .assign(
        SepalRatio=lambda x: x.SepalWidth / x.SepalLength,
        PetalRatio=lambda x: x.PetalWidth / x.PetalLength,
    )
    .plot(kind="scatter", x="SepalRatio", y="PetalRatio")
)

Since a function is passed in, the function is computed on the DataFrame being assigned to. Importantly, this is the DataFrame that’s been filtered to those rows with sepal length greater than 5. The filtering happens first, and then the ratio calculations. This is an example where we didn’t have a reference to the filtered DataFrame available.

The function signature for `assign()` is simply `**kwargs`. The keys are the column names for the new fields, and the values are either a value to be inserted (for example, a `Series` or NumPy array), or a function of one argument to be called on the `DataFrame`. A copy of the original `DataFrame` is returned, with the new values inserted.

The order of `**kwargs` is preserved. This allows for dependent assignment, where an expression later in `**kwargs` can refer to a column created earlier in the same `assign()`.

In [ ]:
dfa = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})

In [ ]:
dfa.assign(C=lambda x: x["A"] + x["B"], D=lambda x: x["A"] + x["C"])

In the second expression, `x['C']` will refer to the newly created column, that’s equal to `dfa['A'] + dfa['B']`.

#### Indexing / selection

The basics of indexing are as follows:

|Operation                     |Syntax         |Result   |
|:-------                      |:-----         |:-----   |
|Select column                 |`df[col]`      |Series   |
|Select row by label           |`df.loc[label]`|Series   |
|Select row by integer location|`df.iloc[loc]` |Series   |
|Slice rows                    |`df[5:10] `    |DataFrame|
|Select rows by boolean vector |`df[bool_vec]` |DataFrame|

Row selection, for example, returns a `Series` whose index is the columns of the `DataFrame`:

In [ ]:
df.loc["b"]

In [ ]:
df.iloc[2]

#### Data alignment and arithmetic

Data alignment between `DataFrame` objects automatically aligns on **both** the columns and the index (row labels)**. Again, the resulting object will have the union of the column and row labels.

In [ ]:
df = pd.DataFrame(np.random.randn(10, 4), columns=["A", "B", "C", "D"])

In [ ]:
df2 = pd.DataFrame(np.random.randn(7, 3), columns=["A", "B", "C"])

In [ ]:
df + df2

When doing an operation between `DataFrame` and `Series`, the default behavior is to align the `Series` **index** on the `DataFrame` **columns**, thus broadcasting row-wise. For example:

In [ ]:
df - df.iloc[0]

Arithmetic operations with scalars operate element-wise:

In [ ]:
df * 5 + 2

In [ ]:
1 / df

In [ ]:
df ** 4

Boolean operators operate element-wise as well:

In [ ]:
df1 = pd.DataFrame({"a": [1, 0, 1], "b": [0, 1, 1]}, dtype=bool)

In [ ]:
df2 = pd.DataFrame({"a": [0, 1, 1], "b": [1, 1, 0]}, dtype=bool)

In [ ]:
df1 & df2

In [ ]:
df1 | df2

In [ ]:
df1 ^ df2

In [ ]:
-df1

#### Transposing

To transpose, access the `T` attribute or `DataFrame.transpose()`, similar to an ndarray:

In [ ]:
df[:5].T

## Data indexing and selection

The axis labeling information in Pandas objects serves many purposes:

- Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display.
- Enables automatic and explicit data alignment.
- Allows intuitive getting and setting of subsets of the data set.

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of Pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

The Python and NumPy indexing operators `[]` and attribute operator `.` provide quick and easy access to Pandas data structures across a wide range of use cases. This makes interactive work intuitive, as there’s little new to learn if you already know how to deal with Python dictionaries and NumPy arrays. However, since the type of the data to be accessed isn’t known in advance, directly using standard operators has some optimization limits. For production code, we recommended that you take advantage of the optimized Pandas data access methods exposed in this chapter.

Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called `chained assignment` and should be avoided.

### Different choices for indexing

Object selection has had a number of user-requested additions in order to support more explicit location-based indexing. Pandas now supports three types of multi-axis indexing.

- `.loc` is primarily label based, but may also be used with a boolean array. `.loc` will raise `KeyError` when the items are not found. Allowed inputs are:
  - A single label, e.g. `5` or `'a'` (Note that `5` is interpreted as a label of the index. This use is not an integer position along the index.).
  - A list or array of labels `['a', 'b', 'c']`.
  - A slice object with labels `'a':'f'` (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index!)
  - A boolean array (any `NA` values will be treated as `False`).
  - A `callable` function with one argument (the calling Series or DataFrame) and that returns valid output for indexing (one of the above).

- `.iloc` is primarily integer position based (from `0` to `length-1` of the axis), but may also be used with a boolean array. `.iloc` will raise `IndexError` if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with Python/NumPy slice semantics). Allowed inputs are:
  - An integer e.g. `5`.
  - A list or array of integers `[4, 3, 0]`.
  - A slice object with ints `1:7`.
  - A boolean array (any `NA` values will be treated as `False`).
  - A `callable` function with one argument (the calling Series or DataFrame) that returns valid output for indexing (one of the above).
- `.loc`, `.iloc`, and also `[]` indexing can accept a `callable` as indexer.

Getting values from an object with multi-axes selection uses the following notation (using `.loc` as an example, but the following applies to `.iloc` as well). Any of the axes accessors may be the null slice `:`. Axes left out of the specification are assumed to be `:`, e.g. `p.loc['a']` is equivalent to `p.loc['a', :]`.

|**Object Type**|**Indexers**                        |
|:--            |:-                                  |
|Series         |`s.loc[indexer]`                    |
|DataFrame      |`df.loc[row_indexer, column_indexer]`|

### Basics

As mentioned when introducing the data structures in the last section, the primary function of indexing with `[]` (a.k.a.` __getitem__` for those familiar with implementing class behavior in Python) is selecting out lower-dimensional slices. The following table shows return type values when indexing Pandas objects with `[]`:

|**Object Type**|**Selection**   |Return Value Type                |
|:-             |:-              |:-                               |
|Series         |`series[label]` |scalar value                     |
|DataFrame      |`frame[colname]`|`Series` corresponding to colname|

Here we construct a simple time series data set to use for illustrating the indexing functionality:

In [ ]:
dates = pd.date_range('1/1/2000', periods=8)
df = pd.DataFrame(np.random.randn(8, 4),
                  index=dates, columns=['A', 'B', 'C', 'D'])
df

None of the indexing functionality is time series specific unless specifically stated.

Thus, as per above, we have the most basic indexing using `[]`:

In [ ]:
s = df['A']

s[dates[5]]

You can pass a list of columns to `[]` to select columns in that order. If a column is not contained in the DataFrame, an exception will be raised. Multiple columns can also be set in this manner:

In [ ]:
df

In [ ]:
df[['B', 'A']] = df[['A', 'B']]
df

You may find this useful for applying a transform (in-place) to a subset of the columns.

Pandas aligns all AXES when setting `Series` and `DataFrame` from `.loc`, and `.iloc`.

This will not modify `df` because the column alignment is before value assignment.

In [ ]:
df[['A', 'B']]

In [ ]:
df.loc[:, ['B', 'A']] = df[['A', 'B']]
df[['A', 'B']]

The correct way to swap column values is by using raw values:

In [ ]:
df.loc[:, ['B', 'A']] = df[['A', 'B']].to_numpy()
df[['A', 'B']]

### Attribute access

You may access an index on a `Series` or column on a `DataFrame` directly as an attribute:

In [ ]:
sa = pd.Series([1, 2, 3], index=list('abc'))
dfa = df.copy()

In [ ]:
sa.b

In [ ]:
dfa.A

In [ ]:
sa.a = 5
sa

In [ ]:
dfa.A = list(range(len(dfa.index)))  # ok if A already exists
dfa

In [ ]:
dfa['A'] = list(range(len(dfa.index)))  # use this form to create a new column
dfa

- You can use this access only if the index element is a valid Python identifier, e.g. s.1 is not allowed. See here for an explanation of valid identifiers.

- The attribute will not be available if it conflicts with an existing method name, e.g. s.min is not allowed, but s['min'] is possible.

- Similarly, the attribute will not be available if it conflicts with any of the following list: index, major_axis, minor_axis, items.

- In any of these cases, standard indexing will still work, e.g. s['1'], s['min'], and s['index'] will access the corresponding element or column.

If you are using the IPython environment, you may also use tab-completion to see these accessible attributes.

You can also assign a `dict` to a row of a `DataFrame`:

In [ ]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
x.iloc[1] = {'x': 9, 'y': 99}
x

You can use attribute access to modify an existing element of a Series or column of a DataFrame, but be careful; if you try to use attribute access to create a new column, it creates a new attribute rather than a new column. In 0.21.0 and later, this will raise a `UserWarning`:

In [ ]:
df = pd.DataFrame({'one': [1., 2., 3.]})
df.two = [4, 5, 6]

In [ ]:
df

### Slicing ranges

For now, we explain the semantics of slicing using the [] operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:

In [ ]:
s

In [ ]:
s[:5]

In [ ]:
s[::2]

In [ ]:
s[::-1]

Note that setting works as well:

In [ ]:
s2 = s.copy()
s2[:5] = 0
s2

With DataFrame, slicing inside of `[]` slices the rows. This is provided largely as a convenience since it is such a common operation.

In [ ]:
df[:3]

In [ ]:
df[::-1]

### Selection by label

Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called `chained assignment` and should be avoided.

`.loc` is strict when you present slicers that are not compatible (or convertible) with the index type. For example using integers in a `DatetimeIndex`. These will raise a `TypeError`.

In [ ]:
dfl = pd.DataFrame(np.random.randn(5, 4),
                   columns=list('ABCD'),
                   index=pd.date_range('20130101', periods=5))

String likes in slicing can be convertible to the type of the index and lead to natural slicing.

In [ ]:
dfl.loc['20130102':'20130104']

Pandas will raise a `KeyError` if indexing with a list with missing labels.

Pandas provides a suite of methods in order to have **purely label-based indexing**. This is a strict inclusion-based protocol. Every label asked for must be in the index, or a `KeyError` will be raised. When slicing, both the start bound **AND** the stop bound are included, if present in the index. Integers are valid labels, but they refer to the label **and not the position**.

- The `.loc` attribute is the primary access method. The following are valid inputs:

- A single label, e.g. `5` or `'a'` (Note that `5` is interpreted as a label of the index. This use is not an integer position along the index.).

- A list or array of labels `['a', 'b', 'c']`.

- A slice object with labels `'a':'f'` (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index!

- A boolean array.

- A `callable`.

In [ ]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))
s1
s1.loc['c':]

In [ ]:
s1.loc['b']

In [ ]:
s1.loc['c':] = 0
s1

In [ ]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))
df1
df1.loc[['a', 'b', 'd'], :]

In [ ]:
df1.loc['d':, 'A':'C']

In [ ]:
df1.loc['a']

In [ ]:
df1.loc['a'] > 0

In [ ]:
df1.loc[:, df1.loc['a'] > 0]

In [ ]:
mask = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")
mask

In [ ]:
df1[mask]

In [ ]:
df1.loc['a', 'A'] # this is also equivalent to ``df1.at['a','A']``

In [ ]:
s = pd.Series(list('abcde'), index=[0, 3, 2, 5, 4])
s.loc[3:5]

In [ ]:
s.sort_index()

In [ ]:
s.sort_index().loc[1:6]

In [ ]:
s = pd.Series(list('abcdef'), index=[0, 3, 2, 5, 4, 2])
s.loc[3:5]

Whether a copy or a reference is returned for a setting operation, may depend on the context. This is sometimes called `chained assignment` and should be avoided.

Pandas provides a suite of methods in order to get purely integer-based indexing. The semantics follow closely Python and NumPy slicing. These are 0-based indexing. When slicing, the start bound is included, while the upper bound is excluded. Trying to use a non-integer, even a valid label will raise an `IndexError`.

The `.iloc` attribute is the primary access method. The following are valid inputs:

- An integer e.g. `5`.

- A list or array of integers `[4, 3, 0]`.

- A slice object with ints `1:7`.

- A boolean array.

- A `callable`.

In [ ]:
s1 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))
s1
s1.iloc[:3]

In [ ]:
s1.iloc[3]

In [ ]:
s1.iloc[:3] = 0
s1

In [ ]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list(range(0, 12, 2)),
                   columns=list(range(0, 8, 2)))
df1
df1.iloc[:3]

In [ ]:
df1.iloc[1:5, 2:4]

In [ ]:
df1.iloc[[1, 3, 5], [1, 3]]

In [ ]:
df1.iloc[1:3, :]

In [ ]:
df1.iloc[:, 1:3]

In [ ]:
df1.iloc[1, 1] # this is also equivalent to ``df1.iat[1,1]``

In [ ]:
df1.iloc[1]

In [ ]:
x = list('abcdef') # these are allowed in Python/NumPy.
x

In [ ]:
x[4:10]

In [ ]:
x[8:10]

In [ ]:
s = pd.Series(x)
s

In [ ]:
s.iloc[4:10]

In [ ]:
s.iloc[8:10]

Note that using slices that go out of bounds can result in an empty axis (e.g. an empty DataFrame being returned).

In [ ]:
dfl = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))

In [ ]:
dfl.iloc[:, 2:3]

In [ ]:
dfl.iloc[:, 1:3]

In [ ]:
dfl.iloc[4:6]

### Selection by callable

`.loc`, `.iloc`, and also `[]` indexing can accept a `callable` as indexer. The `callable` must be a function with one argument (the calling Series or DataFrame) that returns valid output for indexing.

In [ ]:
df1 = pd.DataFrame(np.random.randn(6, 4),
                   index=list('abcdef'),
                   columns=list('ABCD'))
df1
df1.loc[lambda df: df['A'] > 0, :]

In [ ]:
df1.loc[:, lambda df: ['A', 'B']]

In [ ]:
df1.iloc[:, lambda df: [0, 1]]

In [ ]:
df1[lambda df: df.columns[0]]

In [ ]:
df1['A'].loc[lambda s: s > 0]

In [ ]:
dfd = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6]},
                   index=list('abc'))
dfd
dfd.loc[dfd.index[[0, 2]], 'A']

In [ ]:
dfd.iloc[[0, 2], dfd.columns.get_loc('A')]

In [ ]:
dfd.iloc[[0, 2], dfd.columns.get_indexer(['A', 'B'])]

In [ ]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
pd.concat([s1, s2])

Clear the existing index and reset it in the result by setting the `ignore_index` option to `True`.

In [ ]:
pd.concat([s1, s2], ignore_index=True)

Add a hierarchical index at the outermost level of the data with the `keys` option.

In [ ]:
pd.concat([s1, s2], keys=['s1', 's2'])

Label the index keys you create with the `names` option.

In [ ]:
pd.concat([s1, s2], keys=['s1', 's2'],
          names=['Series name', 'Row ID'])

Combine two `DataFrame` objects with identical columns.

In [ ]:
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'])
df1

In [ ]:
df2 = pd.DataFrame([['c', 3], ['d', 4]],
                   columns=['letter', 'number'])
df2

In [ ]:
pd.concat([df1, df2])

Combine `DataFrame` objects with overlapping columns and return everything. Columns outside the intersection will be filled with `NaN` values.

In [ ]:
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']],
                   columns=['letter', 'number', 'animal'])
df3

In [ ]:
pd.concat([df1, df3], sort=False)

Combine DataFrame objects with overlapping columns and return only those that are shared by passing inner to the join keyword argument.

In [ ]:
pd.concat([df1, df3], join="inner")

Combine `DataFrame` objects horizontally along the x-axis by passing in `axis=1`.

In [ ]:
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']],
                   columns=['animal', 'name'])
pd.concat([df1, df4], axis=1)

Prevent the result from including duplicate index values with the `verify_integrity` option.

In [ ]:
df5 = pd.DataFrame([1], index=['a'])
df5

In [ ]:
df6 = pd.DataFrame([2], index=['a'])
df6

In [ ]:
:tags: ["raises-exception"]
pd.concat([df5, df6], verify_integrity=True)

Append a single row to the end of a `DataFrame` object.

In [ ]:
df7 = pd.DataFrame({'a': 1, 'b': 2}, index=[0])
df7

In [ ]:
new_row = pd.Series({'a': 3, 'b': 4})
new_row

In [ ]:
pd.concat([df7, new_row.to_frame().T], ignore_index=True)

`append()` has been deprecated since version 1.4.0: Use `concat()` instead. 

### merge

- Merge DataFrame or named Series objects with a database-style join.

- A named Series object is treated as a DataFrame with a single named column.

- The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on. When performing a cross-merge, no column specifications to merge on are allowed.

If both key columns contain rows where the key is a null value, those rows will be matched against each other. This is different from usual SQL join behaviour and can lead to unexpected results.

For example:

In [ ]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'rkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [5, 6, 7, 8]})

Merge DataFrames `df1` and `df2` with specified left and right suffixes appended to any overlapping columns.

In [ ]:
df1.merge(df2, left_on='lkey', right_on='rkey', suffixes=('_left', '_right'))

Using `how` parameter decide the type of merge to be performed.

In [ ]:
df1 = pd.DataFrame({'a': ['foo', 'bar'], 'b': [1, 2]})
df2 = pd.DataFrame({'a': ['foo', 'baz'], 'c': [3, 4]})

In [ ]:
df1.merge(df2, how='inner', on='a')

In [ ]:
df1.merge(df2, how='left', on='a')

In [ ]:
df1 = pd.DataFrame({'left': ['foo', 'bar']})
df2 = pd.DataFrame({'right': [7, 8]})

In [ ]:
df1.merge(df2, how='cross')

In [ ]:
df = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                   'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})

In [ ]:
other = pd.DataFrame({'key': ['K0', 'K1', 'K2'],
                      'B': ['B0', 'B1', 'B2']})                     

Join DataFrames using their indexes.

In [ ]:
df.join(other, lsuffix='_caller', rsuffix='_other')

In [ ]:
df.set_index('key').join(other.set_index('key'))

In [ ]:
df.join(other.set_index('key'), on='key')

Using non-unique key values shows how they are matched.

In [ ]:
df = pd.DataFrame({'key': ['K0', 'K1', 'K1', 'K3', 'K0', 'K1'],
                   'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})
df                   

In [ ]:
df.join(other.set_index('key'), on='key', validate='m:1')

## Aggregation and grouping

Group `DataFrame` using a mapper or by a `Series` of columns.

A `groupby` operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups.

For example:

In [ ]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
df
df.groupby(['Animal']).mean()

In [ ]:
arrays = [['Falcon', 'Falcon', 'Parrot', 'Parrot'],
          ['Captive', 'Wild', 'Captive', 'Wild']]
index = pd.MultiIndex.from_arrays(arrays, names=('Animal', 'Type'))
df = pd.DataFrame({'Max Speed': [390., 350., 30., 20.]},
                  index=index)
df.groupby(level=0).mean()

In [ ]:
df.groupby(level="Type").mean()

In [ ]:
l = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df = pd.DataFrame(l, columns=["a", "b", "c"])
df.groupby(by=["b"]).sum()

In [ ]:
df.groupby(by=["b"], dropna=False).sum()

In [ ]:
l = [["a", 12, 12], [None, 12.3, 33.], ["b", 12.3, 123], ["a", 1, 1]]
df = pd.DataFrame(l, columns=["a", "b", "c"])
df.groupby(by="a").sum()

In [ ]:
df.groupby(by="a", dropna=False).sum()

In [ ]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
df.groupby("Animal", group_keys=True).apply(lambda x: x)

In [ ]:
df.groupby("Animal", group_keys=False).apply(lambda x: x)

## Pivot table

Create a spreadsheet-style pivot table as a DataFrame.

The levels in the pivot table will be stored in MultiIndex objects (hierarchical indexes) on the index and columns of the result DataFrame.

In [ ]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

This first example aggregates values by taking the sum.

In [ ]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum)
table

We can also fill in missing values using the `fill_value` parameter.

In [ ]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum, fill_value=0)
table

The next example aggregates by taking the mean across multiple columns.

In [ ]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': np.mean})
table

We can also calculate multiple types of aggregations for any given value column.

In [ ]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})
table

## High-performance Pandas: eval() and query()

### eval()

Evaluate a string describing operations on DataFrame columns.

Operates on columns only, not specific rows or elements. This allows `eval` to run arbitrary code, which can make you vulnerable to code injection if you pass user input to this function.

For example:

In [ ]:
df = pd.DataFrame({'A': range(1, 6), 'B': range(10, 0, -2)})
df

In [ ]:
df.eval('A + B')

The assignment is allowed though by default the original `DataFrame` is not modified.

In [ ]:
df.eval('C = A + B')

In [ ]:
df

Use `inplace=True` to modify the original DataFrame.

In [ ]:
df.eval('C = A + B', inplace=True)
df

Multiple columns can be assigned using multi-line expressions:

In [ ]:
df.eval(
    '''
    C = A + B
    D = A - B
    '''
)

### query()

Query the columns of a DataFrame with a boolean expression.

For example:

In [ ]:
df = pd.DataFrame({
    'A': range(1, 6),
    'B': range(10, 0, -2),
    'C C': range(10, 5, -1)
})
df

In [ ]:
df.query('A > B')

The previous expression is equivalent to

In [ ]:
df[df.A > df.B]

For columns with spaces in their name, you can use backtick quoting.

In [ ]:
df.query('B == `C C`')

The previous expression is equivalent to

In [ ]:
df[df.B == df['C C']]


## Your turn! 🚀

### Processing image data

Recently, very powerful AI models have been developed that allow us to understand images. There are many tasks that can be solved using pre-trained neural networks, or cloud services. Some examples include:

- **Image Classification**, can help you categorize the image into one of the pre-defined classes. You can easily train your own image classifiers using services such as [Custom Vision](https://azure.microsoft.com/services/cognitive-services/custom-vision-service/?WT.mc_id=academic-77958-bethanycheum)
- **Object Detection** to detect different objects in the image. Services such as [computer vision](https://azure.microsoft.com/services/cognitive-services/computer-vision/?WT.mc_id=academic-77958-bethanycheum) can detect a number of common objects, and you can train [Custom Vision](https://azure.microsoft.com/services/cognitive-services/custom-vision-service/?WT.mc_id=academic-77958-bethanycheum) model to detect some specific objects of interest.
- **Face Detection**, including Age, Gender and Emotion detection. This can be done via [Face API](https://azure.microsoft.com/services/cognitive-services/face/?WT.mc_id=academic-77958-bethanycheum).

All those cloud services can be called using [Python SDKs](https://docs.microsoft.com/samples/azure-samples/cognitive-services-python-sdk-samples/cognitive-services-python-sdk-samples/?WT.mc_id=academic-77958-bethanycheum), and thus can be easily incorporated into your data exploration workflow.

Here are some examples of exploring data from Image data sources:

- In the blog post [How to Learn Data Science without Coding](https://soshnikov.com/azure/how-to-learn-data-science-without-coding/) we explore Instagram photos, trying to understand what makes people give more likes to a photo. We first extract as much information from pictures as possible using [computer vision](https://azure.microsoft.com/services/cognitive-services/computer-vision/?WT.mc_id=academic-77958-bethanycheum), and then use [Azure Machine Learning AutoML](https://docs.microsoft.com/azure/machine-learning/concept-automated-ml/?WT.mc_id=academic-77958-bethanycheum) to build the interpretable model.
- In [Facial Studies Workshop](https://github.com/CloudAdvocacy/FaceStudies) we use [Face API](https://azure.microsoft.com/services/cognitive-services/face/?WT.mc_id=academic-77958-bethanycheum) to extract emotions from people on photographs from events, in order to try to understand what makes people happy.

### Assignment

[Perform more detailed data study for the challenges above](../../assignments/data-science/data-processing-in-python.md)

## Self study

In this chapter, we've covered many of the basics of using Pandas effectively for data analysis. Still, much has been omitted from our discussion. To learn more about Pandas, we recommend the following resources:

- [Pandas online documentation](http://pandas.pydata.org/): This is the go-to source for complete documentation of the package. While the examples in the documentation tend to be small generated datasets, the description of the options is complete and generally very useful for understanding the use of various functions.

- [*Python for Data Analysis*](http://shop.oreilly.com/product/0636920023784.do) Written by Wes McKinney (the original creator of Pandas), this book contains much more detail on the Pandas package than we had room for in this chapter. In particular, he takes a deep dive into tools for time series, which were his bread and butter as a financial consultant. The book also has many entertaining examples of applying Pandas to gain insight from real-world datasets. Keep in mind, though, that the book is now several years old, and the Pandas package has quite a few new features that this book does not cover (but be on the lookout for a new edition in 2017).

- [Stack Overflow](http://stackoverflow.com/questions/tagged/pandas): Pandas has so many users that any question you have has likely been asked and answered on Stack Overflow. Using Pandas is a case where some Google-Fu is your best friend. Simply go to your favorite search engine and type in the question, problem, or error you're coming across–more than likely you'll find your answer on a Stack Overflow page.

- [Pandas on PyVideo](http://pyvideo.org/search?q=pandas): From PyCon to SciPy to PyData, many conferences have featured tutorials from Pandas developers and power users. The PyCon tutorials in particular tend to be given by very well-vetted presenters.

Using these resources, combined with the walk-through given in this chapter, my hope is that you'll be poised to use Pandas to tackle any data analysis problem you come across!

## Acknowledgments

Thanks for [Pandas user guide](https://pandas.pydata.org/docs/user_guide/index.html). It contributes the majority of the content in this chapter.